In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import vtk
from vtkmodules.util import numpy_support
from data.data import fitTransPipeline
from cfg.configs import Config
from train import load_checkpoint
%load_ext autoreload
%autoreload 2

In [2]:
data_path = "../test/final/data/bc80-45000-down.vts"
vtkSGR = vtk.vtkXMLStructuredGridReader()
vtkSGR.SetFileName(data_path)
vtkSGR.Update()
vtkSG = vtkSGR.GetOutput()
print(vtkSG.GetDimensions())
print(vtkSG.GetExtent())
print(vtkSG.GetBounds())

(88, 51, 21)
(0, 87, 0, 50, 0, 20)
(0.0, 696.0, -250.0, 150.0, 0.7504535071217414, 429.4370158281313)


In [8]:
dims = [21, 51, 88]
z,y,x = np.meshgrid(
  np.linspace(0, 1, dims[0]),
  np.linspace(0, 1, dims[1]),
  np.linspace(0, 1, dims[2]),
  indexing="ij"
)
xyz = np.concatenate([z[...,None], y[...,None], x[...,None]], axis=-1)

In [16]:
class Phys2CompDataset(Dataset):
  def __init__(self, data_path, intrans=None, outtrans=None):
    self.data_path = data_path
    vtkSGR = vtk.vtkXMLStructuredGridReader()
    vtkSGR.SetFileName(data_path)
    vtkSGR.Update()
    vtkSG = vtkSGR.GetOutput()
    # get dimension and swap xyz order to zyx
    self.dims = np.array(vtkSG.GetDimensions())
    tmp = self.dims[0]
    self.dims[0] = self.dims[2]
    self.dims[2] = tmp
    # get phys mesh
    self.phys = numpy_support.vtk_to_numpy(vtkSG.GetPoints().GetData())
    # get comp mesh - regular grid
    self.phys = self.phys.reshape([*self.dims, 3])
    z,y,x = np.meshgrid(
      np.linspace(0, 1, self.dims[0]),
      np.linspace(0, 1, self.dims[1]),
      np.linspace(0, 1, self.dims[2]),
      indexing="ij"
    )
    self.comp = np.concatenate([z[...,None], y[...,None], x[...,None]], axis=-1)
    # preprocessing
    self.phys_prep = torch.Tensor(self.phys.reshape(-1, 3))
    self.comp_prep = torch.Tensor(self.comp.reshape(-1, 3))

    if intrans is not None:
      print("transforming inputs")
      self.phys_prep, self.inpp = fitTransPipeline(self.phys.reshape(-1, 3))
      self.phys_prep = torch.Tensor(self.phys_prep)
    
    if outtrans is not None:
      print("transforming outputs")
      self.comp_prep, self.outpp = fitTransPipeline(self.comp.reshape(-1, 3))
      self.comp_prep = torch.Tensor(self.comp_prep)

  def __len__(self):
    return len(self.phys_prep)

  def __getitem__(self, idx):
    return self.phys_prep[idx], self.comp_prep[idx]
    
ds = Phys2CompDataset(data_path)

NameError: name 'data_path' is not defined

In [2]:
impcfg = Config("nn_vls_imp.yaml")

In [8]:
ds_imp = impcfg.get_dataset()

transforming inputs
transforming outputs


tensor(1.)

In [16]:
pp = ds_imp.inpp.get_params()
std = pp['steps'][0][1]

In [23]:
std.scale_

array([219.87124545])

In [3]:
nncfg = Config("nn_vls_md.yaml")
nncfg_lg = Config("nn_vls_lg.yaml")

In [4]:
model_dir = nncfg.config['model']['log_dir']
ckpt = load_checkpoint(f"{model_dir}/ckpt_7298_-001_8.76e-06", nncfg)
model = ckpt['model']

In [2]:
model_dir = nncfg_lg.config['model']['log_dir']
ckpt = load_checkpoint(f"{model_dir}/ckpt_9748_-001_5.31e-06", nncfg_lg)
model_lg = ckpt['model']

NameError: name 'nncfg_lg' is not defined

In [5]:
example=torch.rand(2, 3)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(f"{model_dir}/traced_7300_1.pt")

In [27]:
ds = nncfg.get_dataset()

transforming inputs


In [32]:
example = torch.rand(2, 3)

In [30]:
pred = model(ds.phys_prep)
pred_lg = model_lg(ds.phys_prep)

In [29]:
for i in range(10):
  print(ds.comp_prep[i], pred[i])

tensor([0., 0., 0.]) tensor([0.0250, 0.0157, 0.0214], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0115]) tensor([0.0204, 0.0149, 0.0265], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0230]) tensor([0.0172, 0.0136, 0.0354], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0345]) tensor([0.0173, 0.0120, 0.0463], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0460]) tensor([0.0184, 0.0109, 0.0565], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0575]) tensor([0.0193, 0.0103, 0.0646], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0690]) tensor([0.0201, 0.0099, 0.0761], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0805]) tensor([0.0194, 0.0099, 0.0870], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0920]) tensor([0.0173, 0.0100, 0.0987], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.1034]) tensor([0.0161, 0.0102, 0.1097], grad_fn=<SelectBackward0>)


In [31]:
for i in range(10):
  print(ds.comp_prep[i], pred_lg[i])

tensor([0., 0., 0.]) tensor([0.0190, 0.0203, 0.0236], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0115]) tensor([0.0185, 0.0174, 0.0252], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0230]) tensor([0.0145, 0.0157, 0.0321], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0345]) tensor([0.0111, 0.0135, 0.0458], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0460]) tensor([0.0099, 0.0125, 0.0554], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0575]) tensor([0.0100, 0.0111, 0.0644], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0690]) tensor([0.0102, 0.0099, 0.0742], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0805]) tensor([0.0097, 0.0095, 0.0850], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.0920]) tensor([0.0084, 0.0101, 0.0988], grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0000, 0.1034]) tensor([0.0087, 0.0093, 0.1067], grad_fn=<SelectBackward0>)
